In [4]:
import numpy as np
import math
import time
import pandas as pd
from tqdm import tqdm
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

import data
import features
from model_xgboost import ModelXGBoost

In [5]:
temp, dico = data.load_input_data('data/train_input.csv')
output = data.load_output_data('data/challenge_output.csv', temp, dico)
all_features = features.load_all_features(dico, temp, remove_useless=True)

100%|████████████████████████████████████████████████████████████████████████████████| 240/240 [00:29<00:00,  8.05it/s]


All the features have been loaded in 29.81 sec
16 useless features have been removed. There are now 29 features for each setting.


In [ ]:
name_features = features.choose_name_features(all_features, '')
name_features

In [ ]:
model = ModelXGBoost(all_features, output)

In [ ]:
output['fields']

In [ ]:
model.load_train_test_set(features_names=name_features, pct_train=0.8, do_print=True)

In [ ]:
model.fit_model('Q_total_AC_kW')

In [ ]:
model.test_model()

In [ ]:
def test_type_feature(t):
    print('-- {} --'.format(t))
    name_features = features.choose_name_features(all_features, t)
    model = ModelXGBoost(all_features, output)

    # Metrics sur toutes les variables à prédire. Moyenne sur N separations train/test
    results = model.cross_validate(features_names=name_features, N=5, pct_train=0.75, do_print=True)

In [ ]:
test_type_feature('')
test_type_feature('_non_int')
test_type_feature('_without_lever')
test_type_feature('_without_lever_non_int')

## Charger le modele sur toutes les données pour générer le fichier de prédiction

In [ ]:
name_features = features.choose_name_features(all_features, '')

model = ModelXGBoost(all_features, output)

models = {}
# Train model on all the data
model.load_train_test_set(name_features, pct_train=1.)

for var in model.output_names:
    print("Learning on '{}'...".format(var))
    model.fit_model(var)
    models[var] = model.model

X_val = features.load_data_features('./data/test_input.csv', name_features, remove_useless=True)

model.save_output(models, './data/test_xgboost.csv', X_val)

Learning on 'office_temperature_degreC'...
Learning on 'Q_total_heating_kW'...
